In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [ ]:
df.head(5)

# Train/Test split

In [19]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Id', 'SalePrice'])
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1095, 79) (365, 79) (1095,) (365,)


# Filling missing values

In [6]:
cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
num_cols = [col for col in X_train.columns if X_train[col].dtype != 'object']

print(cat_cols)
print(num_cols)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'Enclos

In [ ]:
#Print columns with missing values
na_counts = X_train.isna().sum()
print(na_counts[na_counts > 0])

In [ ]:
#Print value distributions for each missing variable to infer missing values in the next cell
missing_vars = [
    "LotFrontage", "Alley", "MasVnrType", "MasVnrArea", "BsmtQual",
    "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", 
    "Electrical", "FireplaceQu", "GarageType", "GarageYrBlt", 
    "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature"
]

for col in missing_vars:
    print(X_train[col].value_counts(dropna=False))
    print("-" * 50)

In [ ]:
# For houses without lot frontage
X_train['LotFrontage'] = X_train['LotFrontage'].fillna(0)

# For houses without an alleyway
X_train['Alley'] = X_train['Alley'].fillna('NoAlley')

# For houses with no masonry work done
X_train['MasVnrArea'] = X_train['MasVnrArea'].fillna(0)
X_train['MasVnrType'] = X_train['MasVnrType'].fillna('NoMasonry')

# For houses with no basement
X_train['BsmtQual'] = X_train['BsmtQual'].fillna('NoBsmtQual')
X_train['BsmtCond'] = X_train['BsmtCond'].fillna('NoBsmtCond')
X_train['BsmtExposure'] = X_train['BsmtExposure'].fillna('NoBsmtExposure')
X_train['BsmtFinType1'] = X_train['BsmtFinType1'].fillna('NoBsmtFinType1')
X_train['BsmtFinType2'] = X_train['BsmtFinType2'].fillna('NoBsmtFinType2')

# For houses without certain features
X_train['Electrical'] = X_train['Electrical'].fillna('NoElectrical')
X_train['FireplaceQu'] = X_train['FireplaceQu'].fillna('NoFireplaceQu')
X_train['PoolQC'] = X_train['PoolQC'].fillna('NoPool')
X_train['Fence'] = X_train['Fence'].fillna('NoFence')
X_train['MiscFeature'] = X_train['MiscFeature'].fillna('NoMisc')

# For houses without a garage
X_train['GarageType'] = X_train['GarageType'].fillna('NoGarage')
X_train['GarageYrBlt'] = X_train['GarageYrBlt'].fillna(0)
X_train['GarageFinish'] = X_train['GarageFinish'].fillna('NoGarageFin')
X_train['GarageQual'] = X_train['GarageQual'].fillna('NoGarageQual')
X_train['GarageCond'] = X_train['GarageCond'].fillna('NoGarageCond')


In [ ]:
print(len(X.columns))
print(len([col for col in X.columns if X[col].dtype == 'object']))
print(len([col for col in X.columns if X[col].dtype != 'object']))

In [10]:
print(y.min())
print(y.max())
#prices look to be in a reasonable range with no apparent outliers

34900
755000


In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.cat_cols = []
        self.num_cols = []
    
    def fit(self, X, y=None):
        self.cat_cols = [col for col in X.columns if X[col].dtype == 'object']
        self.num_cols = [col for col in X.columns if X[col].dtype != 'object']
        return self
    
    def transform(self, X):
        X = X.copy()        
        for col in self.cat_cols:
            X[col] = X[col].fillna(f'No{col}')
        
        for col in self.num_cols:
            X[col] = X[col].fillna(0)
        
        return X

In [ ]:
data_cleaner = DataCleaner()
data_cleaner.fit(X_train)
X_train_cleaned = data_cleaner.transform(X_train)
total_na = X_train_cleaned.isna().sum().sum()
print(f'Total number of missing (NA) values: {total_na}')

# One-hot encoder

In [8]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, one_hot_columns=None):
        self.one_hot_columns = one_hot_columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        
        encoded = pd.get_dummies(X[self.one_hot_columns], drop_first=True)
        X = X.drop(columns=self.one_hot_columns)
        X = pd.concat([X, encoded], axis=1)
        X[encoded.columns] = X[encoded.columns].astype(int)
        
        return X


In [ ]:
encoder = CategoricalEncoder(one_hot_columns=cat_cols)
X_train_encoded = encoder.fit_transform(X_train_cleaned)
print(X_train_encoded.shape)


In [ ]:
cat_cols2 = [col for col in X_train_encoded.columns if X_train_encoded[col].dtype == 'object']
num_cols2 = [col for col in X_train_encoded.columns if X_train_encoded[col].dtype != 'object']

print(cat_cols2)
print(num_cols2)

# Correlation filter

In [9]:
class CorrelationFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.features_to_drop = []

    def fit(self, X, y):
        corr_matrix = X.corr().abs()
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        high_corr_pairs = []

        for i in range(len(corr_matrix.columns)):
            for j in range(i + 1, len(corr_matrix.columns)):
                if corr_matrix.iloc[i, j] > self.threshold:
                    high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
        
        features_to_drop = []
        for feat1, feat2, _ in high_corr_pairs:
            if abs(X[feat1].corr(y)) < abs(X[feat2].corr(y)):
                features_to_drop.append(feat1)
            else:
                features_to_drop.append(feat2)
        
        self.features_to_drop = list(set(features_to_drop))
        return self

    def transform(self, X):
        return X.drop(columns=self.features_to_drop)




In [ ]:
correlation_filter = CorrelationFilter(threshold=0.8)
correlation_filter.fit(X_train_encoded, y_train)
X_train_filtered = correlation_filter.transform(X_train_encoded)

print(X_train_filtered.head())
print(X_train_filtered.shape)

In [ ]:
numeric_cols = X_train_filtered.select_dtypes(include='number')
numeric_count = numeric_cols.shape[1]

non_numeric_cols = X_train_filtered.select_dtypes(exclude='number')
non_numeric_count = non_numeric_cols.shape[1]

print(f'Number of numeric columns: {numeric_count}')
print(f'Number of non-numeric columns: {non_numeric_count}')


# RFE feature selection

In [10]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureSelectorRFE(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=30, estimator=None):
        self.n_features = n_features
        self.estimator = estimator if estimator is not None else DecisionTreeRegressor()
        self.selected_features = None
        
    def fit(self, X, y):
        if self.selected_features is None:
            rfe = RFE(self.estimator, n_features_to_select=self.n_features)
            rfe.fit(X, y)
            self.selected_features = X.columns[rfe.support_].tolist()
        return self

    def transform(self, X):
        return X[self.selected_features]


In [ ]:
from sklearn.tree import DecisionTreeRegressor

feature_selector = FeatureSelectorRFE(n_features=100)
feature_selector.fit(X_train_filtered, y_train)
X_train_selected = feature_selector.transform(X_train_filtered)

print("Selected features:", feature_selector.selected_features)

# Training

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")

Training data shape: (1168, 79), Test data shape: (292, 79)


In [13]:
!pip install dagshub mlflow
import dagshub
dagshub.init(repo_owner='dimna21', repo_name='ML_Assignment1', mlflow=True)

Initialized MLflow to track repo "dimna21/ML_Assignment1"

Repository dimna21/ML_Assignment1 initialized!

In [20]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data_cleaner = DataCleaner()
categorical_encoder = CategoricalEncoder(one_hot_columns=cat_cols)

X = data_cleaner.fit_transform(X)
X = categorical_encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Parameter grids
param_grid_lin = {
    'scaler': scalers,
    'feature_selector__n_features': [10, 20, 50, 100, 150],
}


In [21]:
# Grid Search Setup for Linear Regression
with mlflow.start_run(run_name="Linear_Regression_Experiment"):
    mlflow.set_experiment("Linear_Regression_Experiment")

    # Log param_grid_lin
    mlflow.log_params({
        "param_grid_lin_scalers": str(param_grid_lin['scaler']),
        "param_grid_lin_feature_selector_n_features": str(param_grid_lin['feature_selector__n_features']),
    })

    # Build the pipeline for LinearRegression
    pipeline_lin = Pipeline([
        ('corr_filter', CorrelationFilter()),
        ('feature_selector', FeatureSelectorRFE(estimator=LinearRegression())),
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])

    grid_lin = GridSearchCV(
        pipeline_lin,
        param_grid=param_grid_lin,
        cv=4,
        scoring='r2',
        refit='r2',
        return_train_score=True,
        verbose=2
    )

    # Log experiment parameters
    mlflow.log_params({
        "cv_folds": 4,
        "scoring_metric": "r2",
        "random_state": 42,
        "model_type": "LinearRegression"
    })

    # Fit the grid search
    grid_lin.fit(X_train, y_train)

    best_lin = grid_lin.best_estimator_
    mlflow.log_param("best_params", grid_lin.best_params_)

    # Log train & test metrics (MAE, RMSE, R^2)
    y_train_pred = best_lin.predict(X_train)
    y_test_pred  = best_lin.predict(X_test)

    metrics_lin = {
        "train_MAE": mean_absolute_error(y_train, y_train_pred),
        "train_RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "train_R2": r2_score(y_train, y_train_pred),
        "test_MAE": mean_absolute_error(y_test, y_test_pred),
        "test_RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "test_R2": r2_score(y_test, y_test_pred),
    }
    mlflow.log_metrics(metrics_lin)

    # Save the best model
    mlflow.sklearn.log_model(best_lin, "best_model")

    # Log the cross-validation results
    results_lin = pd.DataFrame(grid_lin.cv_results_)
    results_lin.to_csv("cv_results_lin.csv", index=False)
    mlflow.log_artifact("cv_results_lin.csv")


Fitting 4 folds for each of 15 candidates, totalling 60 fits
[CV] END feature_selector__n_features=10, scaler=StandardScaler(); total time=   2.8s
[CV] END feature_selector__n_features=10, scaler=StandardScaler(); total time=   3.0s
[CV] END feature_selector__n_features=10, scaler=StandardScaler(); total time=   2.9s
[CV] END feature_selector__n_features=10, scaler=StandardScaler(); total time=   2.9s
[CV] END feature_selector__n_features=10, scaler=MinMaxScaler(); total time=   2.9s
[CV] END feature_selector__n_features=10, scaler=MinMaxScaler(); total time=   3.1s
[CV] END feature_selector__n_features=10, scaler=MinMaxScaler(); total time=   2.9s
[CV] END feature_selector__n_features=10, scaler=MinMaxScaler(); total time=   2.9s
[CV] END .......feature_selector__n_features=10, scaler=None; total time=   2.9s
[CV] END .......feature_selector__n_features=10, scaler=None; total time=   3.0s
[CV] END .......feature_selector__n_features=10, scaler=None; total time=   2.9s
[CV] END .......

2025/04/06 17:00:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Linear_Regression_Experiment at: https://dagshub.com/dimna21/ML_Assignment1.mlflow/#/experiments/3/runs/c1e8dd3f70c64bf68caf87b5383b6d1b
🧪 View experiment at: https://dagshub.com/dimna21/ML_Assignment1.mlflow/#/experiments/3


In [25]:
# Grid Search Setup for Decision Tree Regressor
param_grid_tree = {
    'scaler': [StandardScaler(), MinMaxScaler(), None],
    'feature_selector__n_features': [10, 20, 50, 150],
    'regressor__min_samples_leaf': [3, 5, 10],
    'regressor__min_samples_split': [5, 10, 20],
    'regressor__max_depth': [5, 10, 20],
}

with mlflow.start_run(run_name="Decision_Tree_Experiment"):
    mlflow.set_experiment("Decision_Tree_Experiment")

    # Log param_grid_tree
    mlflow.log_params({
        "param_grid_tree_scalers": str(param_grid_tree['scaler']),
        "param_grid_tree_feature_selector_n_features": str(param_grid_tree['feature_selector__n_features']),
        "param_grid_tree_min_samples_leaf": str(param_grid_tree['regressor__min_samples_leaf']),
        "param_grid_tree_min_samples_split": str(param_grid_tree['regressor__min_samples_split']),
        "param_grid_tree_max_depth": str(param_grid_tree['regressor__max_depth']),
    })

    pipeline_tree = Pipeline([
        ('corr_filter', CorrelationFilter()),
        ('feature_selector', FeatureSelectorRFE(estimator=DecisionTreeRegressor())),
        ('scaler', StandardScaler()),  # placeholder, will be overridden by grid
        ('regressor', DecisionTreeRegressor(random_state=42))
    ])
    grid_tree = GridSearchCV(
        pipeline_tree,
        param_grid=param_grid_tree,
        cv=4,
        scoring='r2',
        refit='r2',
        return_train_score=True,
        verbose=2
    )

    mlflow.log_params({
        "cv_folds": 4,
        "scoring_metric": "r2",
        "random_state": 42,
        "model_type": "DecisionTreeRegressor"
    })

    # Fit the grid search
    grid_tree.fit(X_train, y_train)

    best_tree = grid_tree.best_estimator_
    mlflow.log_param("best_params", grid_tree.best_params_)

    y_train_pred_tree = best_tree.predict(X_train)
    y_test_pred_tree = best_tree.predict(X_test)

    metrics_tree = {
        "train_MAE": mean_absolute_error(y_train, y_train_pred_tree),
        "train_RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_tree)),
        "train_R2": r2_score(y_train, y_train_pred_tree),
        "test_MAE": mean_absolute_error(y_test, y_test_pred_tree),
        "test_RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_tree)),
        "test_R2": r2_score(y_test, y_test_pred_tree),
    }
    mlflow.log_metrics(metrics_tree)

    mlflow.sklearn.log_model(best_tree, "best_model")

Fitting 4 folds for each of 324 candidates, totalling 1296 fits
[CV] END feature_selector__n_features=10, regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5, scaler=StandardScaler(); total time=   5.0s
[CV] END feature_selector__n_features=10, regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5, scaler=StandardScaler(); total time=   5.1s
[CV] END feature_selector__n_features=10, regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5, scaler=StandardScaler(); total time=   5.2s
[CV] END feature_selector__n_features=10, regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5, scaler=StandardScaler(); total time=   5.1s
[CV] END feature_selector__n_features=10, regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5, scaler=MinMaxScaler(); total time=   5.0s
[CV] END feature_selector__n_features=10, regressor__max_depth=5, regress

2025/04/06 20:57:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Decision_Tree_Experiment at: https://dagshub.com/dimna21/ML_Assignment1.mlflow/#/experiments/4/runs/7702289a9828469bad55790437f20e5b
🧪 View experiment at: https://dagshub.com/dimna21/ML_Assignment1.mlflow/#/experiments/4
